In [6]:
import numpy as np
import csv

In [52]:
class Sudoku:
    
    board = None
    
    def __init__(self):
        
        self.board = np.zeros((9,9))
        with open("Sudoku.csv") as csv_file:
            csv_reader = csv.reader(csv_file,delimiter = ',')
            
            for i,row in enumerate(csv_reader):
                for j,number in enumerate(row[1:]):
                    if number != '':
                        self.board[i,j] = number

    def printBoard(self):
        
        for i in range(0,9):
            print self.board[i,0:3], "|",self.board[i,3:6],"|",self.board[i,6:9]
            if (i+1)%3 == 0 and i!= 8:
                print "------------------------------------"
    
    
game = Sudoku()
game.printBoard()

[0. 1. 0.] | [7. 9. 0.] | [5. 4. 0.]
[0. 0. 0.] | [0. 6. 0.] | [0. 0. 0.]
[0. 6. 0.] | [0. 0. 4.] | [0. 9. 0.]
------------------------------------
[2. 0. 0.] | [5. 0. 0.] | [0. 0. 1.]
[0. 0. 9.] | [0. 0. 0.] | [0. 0. 0.]
[7. 0. 0.] | [0. 0. 0.] | [8. 2. 5.]
------------------------------------
[0. 0. 0.] | [0. 0. 0.] | [0. 1. 0.]
[0. 4. 1.] | [0. 0. 2.] | [6. 0. 0.]
[5. 0. 0.] | [3. 0. 0.] | [0. 0. 0.]
